<a href="https://colab.research.google.com/github/himanshu-nakrani/TMLC-Gen-ai-projects/blob/main/Langchain_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain cohere langchain_community google-search-results

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=ef9322ed82b83acb0fc61625e70e590e754bb1a97f0c3313291ac51e6e50eb61
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstal

In [16]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain_community.llms import Cohere
from langchain import LLMMathChain
from google.colab import userdata
from langchain.llms import GooglePalm

In [19]:
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["SERPAPI_API_KEY"] = userdata.get('serp')

In [22]:
# model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

react_agent_llm = Cohere(cohere_api_key=userdata.get('COHERE_KEY'))
math_chain_llm = Cohere(cohere_api_key=userdata.get('COHERE_KEY'))

tools = load_tools(["serpapi"], llm=react_agent_llm)

llm_math_chain = LLMMathChain.from_llm(llm=math_chain_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)

In [23]:
react_agent = initialize_agent(tools,
                               react_agent_llm,
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True,
                               )

In [27]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of ["Search", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

react_agent.agent.llm_chain.prompt.template=prompt_template

In [28]:
react_agent("When did the last Fifa world cup happen and who won it?", )



> Entering new AgentExecutor chain...
 Thought: I need to search for the last FIFA World Cup winner on the internet since it was held recently
Action: Search
Action Input: last FIFA world cup winner
Observation: [{'title': 'Intercontinental Cup and Club World Cup winners', 'link': 'https://www.fourfourtwo.com/features/intercontinental-cup-and-club-world-cup-winners', 'source': 'FourFourTwo', 'date': '12 hours ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQqhJQ6PnBEIs-tGWyAuLm1urfHcyF8Lywo8eUc2j04QDSAL4etBXB8QU&usqp=CAI&s'}, {'title': 'Iniesta, Kroos, Morgan and more: The World Cup winners who retired in 2024', 'link': 'https://www.fifa.com/en/news/articles/world-cup-winners-retired-2024', 'source': 'FIFA', 'date': '2 days ago'}, {'title': 'CONMEBOL 2024 year in review', 'link': 'https://www.fifa.com/en/articles/conmebol-2024-year-review', 'source': 'FIFA', 'date': '1 day ago'}]
Thought: Thought: The winning country of the last FIFA World Cup was Spain and

{'input': 'When did the last Fifa world cup happen and who won it?',
 'output': 'The last FIFA World Cup was in 2022 and was won by Argentina!'}